In [1]:
import os
import pandas as pd
import ee
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
import numpy as np
import rasterio as rio
import geemap

In [2]:
# # Create datacube
# datacube = xr.Dataset(
#     {
#         "sentinel1_vv": (["time", "y", "x"], sentinel1_vv_data),
#         "sentinel1_vh": (["time", "y", "x"], sentinel1_vh_data),
#         "sentinel2_ndvi": (["time", "y", "x"], sentinel2_ndvi_data),
#         "temperature": (["time", "y", "x"], temperature_data),
#         "elevation": (["y", "x"], elevation_data),
#     },
#     coords={
#         "time": time_coords,
#         "y": y_coords,
#         "x": x_coords,
#     },
# )

# # Save datacube
# datacube.to_netcdf("unified_datacube.nc")

In [3]:
# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize()

# # Define AOI
# aoi = ee.Geometry.Polygon([[
#     [36.306872504019424, -0.5952725839718056],
#     [36.306872504019424, -0.7423752404067301],
#     [36.45206844566468, -0.7423752404067301],
#     [36.45206844566468, -0.5952725839718056],
#     [36.306872504019424, -0.5952725839718056]
# ]])

# # Define time range (last 3 months)
# start_date = '2024-12-01'
# end_date = '2025-03-02'

# # Load Sentinel-1 data
# sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
#     .filterBounds(aoi) \
#     .filterDate(start_date, end_date) \
#     .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
#     .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))

# # Select first image (or mosaic multiple images)
# sentinel1_image = sentinel1.first()

# # Export to GeoTIFF or visualize
# geemap.download_ee_image(sentinel1_image.select(['VV', 'VH']), scale=10, region=aoi, filename='C:/Users/LOCATEG5/amini_technical_assignment/data/sentinel1.tif')

In [4]:
# Initialize the Earth Engine API
ee.Initialize()

# Define the AOI using the provided coordinates
aoi_geometry = ee.Geometry.Polygon([
    [ [36.306872504019424, 0.5952725839718056],
      [36.306872504019424, 0.7423752404067301],
      [36.45206844566468, 0.7423752404067301],
      [36.45206844566468, 0.5952725839718056],
      [36.306872504019424, 0.5952725839718056] ]
])

# Show AOI for verification (in GEE, you can visualize using GEE code editor or folium)
# print(aoi_geometry)

### Extract Sentinel-1 (VV, VH), Sentinel-2 NDVI, Temperature, and Elevation Data:

In [5]:
# Load Sentinel-1 VV and VH data for the past three months
sentinel1_vv = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi_geometry) \
    .filterDate('2024-12-01', '2025-03-02') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV')

sentinel1_vh = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi_geometry) \
    .filterDate('2024-12-01', '2025-03-02') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .select('VH')

In [6]:
# Compute monthly mean for both VV and VH
def monthly_mean(image_collection):
    return image_collection.reduce(ee.Reducer.mean())

vv_monthly_mean = monthly_mean(sentinel1_vv)
vh_monthly_mean = monthly_mean(sentinel1_vh)

In [7]:
# !pip install geedim

In [8]:
# Load MODIS temperature data (MOD11A2) for the past three months
temperature = ee.ImageCollection('MODIS/061/MOD11A2') \
    .filterBounds(aoi_geometry) \
    .filterDate('2024-12-01', '2025-03-02') \
    .select('LST_Day_1km')  # Daytime Land Surface Temperature

# Apply scale factor and convert to Celsius
def scale_and_convert(image):
    # Scale the temperature using the scale factor (0.02)
    temp_kelvin = image.multiply(0.02)
    
    # Convert temperature from Kelvin to Celsius
    temp_celsius = temp_kelvin.subtract(273.15)
    
    return temp_celsius.rename('Temperature_Celsius')

# Map the function to the image collection
temperature_celsius = temperature.map(scale_and_convert)

# Compute monthly mean for temperature in Celsius
temperature_monthly_mean = monthly_mean(temperature_celsius)

In [9]:
# Load elevation data (SRTM)
elevation = ee.Image('USGS/SRTMGL1_003') \
    .clip(aoi_geometry)  # Clip to the AOI

In [10]:
# Load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED');

# Filter the Sentinel-2 collection
filtered_s2 = (s2
               .filterDate('2024-12-01', '2025-03-02')
               .filterBounds(aoi_geometry)
               .sort('CLOUDY_PIXEL_PERCENTAGE', True))

In [11]:
monthly_filtered_s2 = monthly_mean(filtered_s2)

In [12]:
monthly_filtered_s2

In [13]:
# Function to compute spectral indices
def ndvi(image):
    ndvi = image.normalizedDifference(['B8_mean', 'B4_mean']).rename('ndvi')
    return image.addBands(ndvi)

# Compute NDVI
s2_composite = ndvi(monthly_filtered_s2)
ndvi = s2_composite.select('ndvi')
ndvi

In [14]:
Map = geemap.Map(center=[0.7423752404067301, 36.45206844566468], zoom=6)
vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'yellow', 'green', 'darkgreen'],
}
Map.addLayer(ndvi.clip(aoi_geometry), vis_params, 'ndvi')
Map

Map(center=[0.7423752404067301, 36.45206844566468], controls=(WidgetControl(options=['position', 'transparent_…

In [15]:
ndvi_aoi = ndvi.clip(aoi_geometry)

In [16]:
# Export Sentinel-1 VV and VH to GeoTIFF
# geemap.download_ee_image(vv_monthly_mean, scale=10, crs='EPSG:4326', region=aoi_geometry, filename='C:/Users/LOCATEG5/amini_technical_assignment/data/sentinel1_vv.tif')
# geemap.download_ee_image(vh_monthly_mean, scale=10, crs='EPSG:4326', region=aoi_geometry, filename='C:/Users/LOCATEG5/amini_technical_assignment/data/sentinel1_vh.tif')
# geemap.download_ee_image(ndvi_aoi, scale=10, crs='EPSG:4326', region=aoi_geometry, filename='C:/Users/LOCATEG5/amini_technical_assignment/data/sentinel2_ndvi.tif')
# geemap.download_ee_image(temperature_monthly_mean, scale=1000, crs='EPSG:4326', region=aoi_geometry, filename='C:/Users/LOCATEG5/amini_technical_assignment/data/temperature.tif')
# geemap.download_ee_image(elevation, scale=30, crs='EPSG:4326', region=aoi_geometry, filename='C:/Users/LOCATEG5/amini_technical_assignment/data/elevation.tif')

### Loading Downloaded Tif files and Resampling them in a the same spatial resolution.

In [17]:
# Load reference dataset (ndvi 10m resolution)
s2_ndvi = rxr.open_rasterio("C:/Users/LOCATEG5/amini_technical_assignment/data/downloaded/sentinel2_ndvi.tif")

# Load and reproject other datasets to match ndvi
s1_vv = rxr.open_rasterio("C:/Users/LOCATEG5/amini_technical_assignment/data/downloaded/sentinel1_vv.tif").rio.reproject_match(s2_ndvi)
s1_vh = rxr.open_rasterio("C:/Users/LOCATEG5/amini_technical_assignment/data/downloaded/sentinel1_vh.tif").rio.reproject_match(s2_ndvi)
temperature = rxr.open_rasterio("C:/Users/LOCATEG5/amini_technical_assignment/data/downloaded/temperature.tif").rio.reproject_match(s2_ndvi)
elevation = rxr.open_rasterio("C:/Users/LOCATEG5/amini_technical_assignment/data/downloaded/elevation.tif").rio.reproject_match(s2_ndvi)

# Ensure all datasets have the same dimensions (width, height)
if not (s1_vv.shape == s1_vh.shape == s2_ndvi.shape == temperature.shape == elevation.shape):
    print("Warning: Some datasets may have mismatched dimensions!")

# Stack all rasters into a single multiband raster (concatenating along band dimension)
multiband = xr.concat([s1_vv, s1_vh, s2_ndvi, temperature, elevation], dim="band")

# Assign band names
multiband = multiband.assign_coords(band=["sentinel-1_vv", "sentinel-1_vh", "sentinel-2_ndvi", "temperature", "elevation"])

# Save as Cloud GeoTIFF(COG)
multiband.rio.to_raster("C:/Users/LOCATEG5/amini_technical_assignment/data/processed/environmental_multiband_cog.tiff")

print("Multiband raster created successfully!")

Multiband raster created successfully!


### Creating Geospatial Raster Datacube

In [18]:
y_coords = s2_ndvi.y.values # Extract latitude or northing values
x_coords = s2_ndvi.x.values # Extract longitude or easting values
# time_coords = pd.date_range(start="2024-12-01", periods=3, freq="M").to_numpy()  # Extract time values

### Now let's create the cube

In [19]:
# Ensure all have the same dimensions
if not (s1_vv.shape == s1_vh.shape == s2_ndvi.shape == temperature.shape == elevation.shape):
    print("Warning: Some datasets may have mismatched dimensions!")

In [20]:
# Assign CRS (EPSG:4326) to all datasets
for ds in [s1_vv, s1_vh, s2_ndvi, temperature, elevation]:
    ds.rio.write_crs("EPSG:4326", inplace=True)

# Stack rasters into an xarray DataArray (Band, Height, Width)
datacube = xr.DataArray(
    data=[s1_vv[0], s1_vh[0], s2_ndvi[0], temperature[0], elevation[0]],  # Extract first band (if needed)
    dims=("band", "y", "x"),
    coords={
        "band": ["sentinel1_vv", "sentinel1_vh", "sentinel-2_ndvi", "temperature", "elevation"], 
        "y": s2_ndvi.y, 
        "x": s2_ndvi.x, 
    },
    attrs={
        "description": "Geospatial Datacube for Africa Environmental Land Monitoring @amini-africa-environmental-data-infrastructure 2025", 
        "crs": "EPSG:4326"
          }
)

# Convert to xarray Dataset (for better metadata handling)
datacube_ds = datacube.to_dataset(name="geospatial_datacube")

# Assign spatial transform (affine transformation) for correct georeferencing
datacube_ds.rio.write_crs("EPSG:4326", inplace=True)
datacube_ds.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
datacube_ds.rio.write_transform(s2_ndvi.rio.transform(), inplace=True)

# Output file paths
output_folder = "C:/Users/LOCATEG5/amini_technical_assignment/data/processed"
netcdf_path = os.path.join(output_folder, "africa_environmental_geospatial_datacube.nc")
zarr_path = os.path.join(output_folder, "africa_environmental_geospatial_datacube.zarr")

# Save as NetCDF
datacube_ds.to_netcdf(netcdf_path)
print(f"Datacube saved as NetCDF: {netcdf_path}")

# Save as Zarr
datacube_ds.to_zarr(zarr_path, mode="w")
print(f"Datacube saved as Zarr: {zarr_path}")

print("Geospatial Datacubes created successfully with 5 bands 👏")

Datacube saved as NetCDF: C:/Users/LOCATEG5/amini_technical_assignment/data/processed\africa_environmental_geospatial_datacube.nc
Datacube saved as Zarr: C:/Users/LOCATEG5/amini_technical_assignment/data/processed\africa_environmental_geospatial_datacube.zarr
Geospatial Datacubes created successfully with 5 bands 👏


In [21]:
# !pip install zarr

In [22]:
datacube_ds

<xarray.Dataset> Size: 106MB
Dimensions:              (band: 5, y: 1639, x: 1617)
Coordinates:
  * band                 (band) <U15 300B 'sentinel1_vv' ... 'elevation'
  * y                    (y) float64 13kB 0.7424 0.7423 0.7422 ... 0.5954 0.5953
  * x                    (x) float64 13kB 36.31 36.31 36.31 ... 36.45 36.45
    spatial_ref          int32 4B 0
Data variables:
    geospatial_datacube  (band, y, x) float64 106MB -5.357 -5.058 ... 1.994e+03

## ..........................................STAC Catalog Creation...................................................

In [23]:
## To come next
import os
import pystac
import rasterio
from datetime import datetime, timezone
from shapely.geometry import box, mapping

In [24]:
# Define paths
output_dir = r"C:\Users\LOCATEG5\amini_technical_assignment\data\processed"
os.makedirs(output_dir, exist_ok=True)


zarr_path = os.path.join(output_dir, "africa_environmental_geospatial_datacube.zarr")
stac_json_path = os.path.join(output_dir, "stac_catalog")

In [25]:
# Read spatial metadata from Zarr using Rasterio
with rasterio.open(os.path.join(zarr_path, "datacube.tiff")) as src:
    bounds = src.bounds  # Get dataset extent
    bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
    footprint = mapping(box(*bbox))  # Convert to GeoJSON polygon

RasterioIOError: C:/Users/LOCATEG5/amini_technical_assignment/data/processed/africa_environmental_geospatial_datacube.zarr/datacube.tiff: No such file or directory

In [ ]:
# Create a STAC Catalog
catalog = pystac.Catalog(
    id="africa_environmental-datacube-catalog",
    description="STAC Catalog for Africa Environmental Data Infrastructure",
    catalog_type=pystac.CatalogType.SELF_CONTAINED
)

In [ ]:
# Set self_href for the catalog
catalog.normalize_hrefs(stac_json_path)

In [ ]:
# Create a STAC Item
item = pystac.Item(
    id="optimized_environmental_infrastructure-datacube",
    geometry=footprint,
    bbox=bbox,
    datetime=datetime.utcnow(),
    properties={"resolution": 10, "crs": "EPSG:4326"}
)

In [ ]:
# Set self_href for the item
item.set_self_href(os.path.join(stac_json_path, "item.json"))

# Add Zarr file as an asset
item.add_asset(
    "zarr",
    pystac.Asset(
        href=zarr_path,
        media_type="application/x-zarr",
        roles=["data"],
        title="Optimized Environmental Infrastructure Datacube"
    )
)

In [ ]:
# Add item to catalog
catalog.add_item(item)

In [ ]:
# Save STAC Catalog to a JSON file
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED, dest_href=stac_json_path)

print(f"STAC catalog successfully saved at: {stac_json_path}")